In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Activation
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Using TensorFlow backend.


In [2]:
# TBD different random seeds
np.random.seed(7)

### On statefulness

Making a RNN stateful means that the states for the samples of each batch will be reused as initial states for the samples in the next batch.

When using stateful RNNs, it is therefore assumed that:

- all batches have the same number of samples
- if X1 and X2 are successive batches of samples, then X2[i] is the follow-up sequence to X1[i], for every i.

Notes that the methods predict, fit, train_on_batch, predict_classes, etc. will all update the states of the stateful layers in a model. This allows you to do not only stateful training, but also stateful prediction.




In [3]:
# whether to use LSTM or MLP
use_LSTM = True

# number of features used in the regression (for MLP)
mlp_num_features = 10
#

# predict several timesteps at once
lstm_predict_sequences = True
lstm_num_predictions = 5

# lstm_num_timesteps
lstm_num_timesteps = 5
# lstm_num_features
lstm_num_features = 1
# stateful?
lstm_stateful = False
# use two lstm layers?
lstm_stack_layers = False

# window_size
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features

batch_size = 1
num_epochs = 500
# dimensionality of the output space
num_neurons = 4

# scale the dataset to values between scale_min and scale_max
scale = False
scale_min = -1
scale_max = 1
#scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
scaler = StandardScaler()

In [4]:
# various test datasets
ts_train_lineartrend = np.arange(1,101, dtype='float64').reshape(-1,1)  
ts_test_lineartrend_outofrange = np.arange(101,121, dtype='float64').reshape(-1,1)
ts_test_lineartrend_withinrange = np.arange(21,41, dtype='float64').reshape(-1,1)


In [5]:
testname = 'linear_trend_within_range'
ts_train = ts_train_lineartrend
ts_test = ts_test_lineartrend_withinrange
ts_all = np.append(ts_train, ts_test).reshape(-1,1)
len_overall = len(ts_all)

In [6]:
len_overall

120

In [7]:
ts_all.dtype

dtype('float64')

In [8]:
ts_train.shape, ts_test.shape

((100, 1), (20, 1))

In [9]:
ts_train[:10]

array([[  1.],
       [  2.],
       [  3.],
       [  4.],
       [  5.],
       [  6.],
       [  7.],
       [  8.],
       [  9.],
       [ 10.]])

In [10]:
ts_test[:10]

array([[ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.],
       [ 30.]])

In [11]:
if scale:
    ts_train = scaler.fit_transform(ts_train)
    ts_test = scaler.transform(ts_test)

In [12]:
ts_train[:10]

array([[  1.],
       [  2.],
       [  3.],
       [  4.],
       [  5.],
       [  6.],
       [  7.],
       [  8.],
       [  9.],
       [ 10.]])

In [13]:
ts_test[:10]

array([[ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.],
       [ 30.]])

In [14]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size), 0]
        dataX.append(a)
        dataY.append(dataset[i + window_size, 0])
    return np.array(dataX), np.array(dataY)

In [15]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

In [16]:
if use_LSTM:
    if lstm_predict_sequences:
        X_train, y_train = create_dataset2(ts_train, lstm_num_timesteps)
        X_test, y_test = create_dataset2(ts_test, lstm_num_timesteps)
    else:    
        X_train, y_train = create_dataset(ts_train, lstm_num_timesteps)
        X_test, y_test = create_dataset(ts_test, lstm_num_timesteps)
else:
    X_train, y_train = create_dataset(ts_train, mlp_num_features)
    X_test, y_test = create_dataset(ts_test, mlp_num_features)
    
# the train and test matrices end up shorter than the respective timeseries by window_size + 1!
X_train.shape, y_train.shape, X_test.shape,  y_test.shape

((90, 5), (90, 5), (10, 5), (10, 5))

In [17]:
X_train[:5,:]

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 2.,  3.,  4.,  5.,  6.],
       [ 3.,  4.,  5.,  6.,  7.],
       [ 4.,  5.,  6.,  7.,  8.],
       [ 5.,  6.,  7.,  8.,  9.]])

In [18]:
y_train[:5,:]

array([[  6.,   7.,   8.,   9.,  10.],
       [  7.,   8.,   9.,  10.,  11.],
       [  8.,   9.,  10.,  11.,  12.],
       [  9.,  10.,  11.,  12.,  13.],
       [ 10.,  11.,  12.,  13.,  14.]])

In [19]:
X_test[:5,:]

array([[ 21.,  22.,  23.,  24.,  25.],
       [ 22.,  23.,  24.,  25.,  26.],
       [ 23.,  24.,  25.,  26.,  27.],
       [ 24.,  25.,  26.,  27.,  28.],
       [ 25.,  26.,  27.,  28.,  29.]])

In [20]:
y_test[:5,:]

array([[ 26.,  27.,  28.,  29.,  30.],
       [ 27.,  28.,  29.,  30.,  31.],
       [ 28.,  29.,  30.,  31.,  32.],
       [ 29.,  30.,  31.,  32.,  33.],
       [ 30.,  31.,  32.,  33.,  34.]])

In [21]:
if use_LSTM:
    # reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    
    if lstm_predict_sequences:
        y_train = np.reshape(y_train, (y_train.shape[0], lstm_num_predictions, lstm_num_features))
        y_test = np.reshape(y_test, (y_test.shape[0], lstm_num_predictions, lstm_num_features))

In [22]:
X_train.shape, y_train.shape, X_test.shape,  y_test.shape

((90, 5, 1), (90, 5, 1), (10, 5, 1), (10, 5, 1))

In [23]:
model = Sequential()

# LSTM input shape
# (samples, time steps, features)
# LSTM output shape
# if return_sequences: 3D tensor with shape (batch_size, timesteps, units).
# else, 2D tensor with shape (batch_size, units).

if use_LSTM:
    
    print('LSTM')
    # the last state for each sample at index i in a batch will be used as initial state
    # for the sample of index i in the following batch
    if lstm_stateful:
        print('stateful')
        #
        if lstm_stack_layers:
            print('stack_layers')
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            print(model.output_shape)
            model.add(LSTM(num_neurons,
                       stateful = True))
            print(model.output_shape)
            model.add(Dense(1))
            print(model.output_shape)
            model.compile(loss='mean_squared_error', optimizer='adam')
            
        # 
        elif lstm_predict_sequences:
            print('predict_sequences')
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            print(model.output_shape)
            model.add(TimeDistributed(Dense(1)))
            print(model.output_shape)
            model.add(Activation("linear"))  
            model.compile(loss='mean_squared_error', optimizer='adam')
            
        #    
        else:
            print('predict single')
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True))
            print(model.output_shape)
            model.add(Dense(1))
            print(model.output_shape)
            model.compile(loss='mean_squared_error', optimizer='adam')
        
    

    # stateful == False    
    else: 
        print('stateless')
        
        if lstm_stack_layers:
            print('stack layers')
            # input_dim: dimensionality of the input (alternatively, input_shape)
            # required when using this layer as the first layer in a model
            model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True))
            print(model.output_shape)
            model.add(LSTM(num_neurons))
            print(model.output_shape)
            model.add(Dense(1))
            print(model.output_shape)
            model.compile(loss='mean_squared_error', optimizer='adam')
        # 
        # 
        elif lstm_predict_sequences:
            print('predict_sequences')
            model.add(LSTM(num_neurons,
                      #  input_dim = lstm_num_features,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]), 
                       return_sequences = True))
            print(model.output_shape) 
            model.add(TimeDistributed(Dense(1)))
            print(model.output_shape) 
            model.compile(loss='mean_squared_error', optimizer='adam')
            
        else:
            print('predict single')
            model.add(LSTM(num_neurons, input_dim = lstm_num_features))
            print(model.output_shape) 
            model.add(Dense(1))
            print(model.output_shape) 
            model.compile(loss='mean_squared_error', optimizer='adam')
        
   

# feedforward
else:
    print('MLP')
    
    model.add(Dense(num_neurons, input_dim = mlp_num_features, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')


LSTM
stateless
predict_sequences
(1, 5, 4)
(1, 5, 1)


In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 5, 4)                 96        
_________________________________________________________________
time_distributed_1 (TimeDist (1, 5, 1)                 5         
Total params: 101.0
Trainable params: 101
Non-trainable params: 0.0
_________________________________________________________________


In [25]:

if use_LSTM & lstm_stateful:
        
    for i in range(num_epochs):
            print('epoch: ' + str(i))
            # shuffle must be False!
            model.fit(X_train, y_train, epochs = 1, batch_size = batch_size, shuffle = False)
            model.reset_states()
            
else: 
    model.fit(X_train, y_train, epochs = num_epochs, batch_size = batch_size)


Epoch 1/500
90/90 [==============================] - 0s - loss: 3374.4669     
Epoch 2/500
90/90 [==============================] - 0s - loss: 3317.5146     
Epoch 3/500
90/90 [==============================] - 0s - loss: 3288.9500     
Epoch 4/500
90/90 [==============================] - 0s - loss: 3263.7686     
Epoch 5/500
90/90 [==============================] - 0s - loss: 3239.5281     
Epoch 6/500
90/90 [==============================] - 0s - loss: 3215.6521     
Epoch 7/500
90/90 [==============================] - 0s - loss: 3174.5908     
Epoch 8/500
90/90 [==============================] - 0s - loss: 3118.5414     
Epoch 9/500
90/90 [==============================] - 0s - loss: 3083.1171     
Epoch 10/500
90/90 [==============================] - 0s - loss: 3050.5697     
Epoch 11/500
90/90 [==============================] - 0s - loss: 3012.3502     
Epoch 12/500
90/90 [==============================] - 0s - loss: 2967.8467     
Epoch 13/500
90/90 [=============================

90/90 [==============================] - 0s - loss: 1180.3296     
Epoch 104/500
90/90 [==============================] - 0s - loss: 1167.6686     
Epoch 105/500
90/90 [==============================] - 0s - loss: 1155.2687     
Epoch 106/500
90/90 [==============================] - 0s - loss: 1142.9160     
Epoch 107/500
90/90 [==============================] - 0s - loss: 1130.6791     
Epoch 108/500
90/90 [==============================] - 0s - loss: 1118.5391     
Epoch 109/500
90/90 [==============================] - 0s - loss: 1106.5549     
Epoch 110/500
90/90 [==============================] - 0s - loss: 1094.5436     
Epoch 111/500
90/90 [==============================] - 0s - loss: 1082.8524     
Epoch 112/500
90/90 [==============================] - 0s - loss: 1071.1592     
Epoch 113/500
90/90 [==============================] - 0s - loss: 1059.5869     
Epoch 114/500
90/90 [==============================] - 0s - loss: 1048.2096     
Epoch 115/500
90/90 [=====================

90/90 [==============================] - 0s - loss: 359.0157     
Epoch 206/500
90/90 [==============================] - 0s - loss: 354.4918     
Epoch 207/500
90/90 [==============================] - 0s - loss: 349.9070     
Epoch 208/500
90/90 [==============================] - 0s - loss: 345.5274     
Epoch 209/500
90/90 [==============================] - 0s - loss: 341.3036     
Epoch 210/500
90/90 [==============================] - 0s - loss: 336.9153     
Epoch 211/500
90/90 [==============================] - 0s - loss: 332.4666     
Epoch 212/500
90/90 [==============================] - 0s - loss: 328.1586     
Epoch 213/500
90/90 [==============================] - 0s - loss: 324.0375     
Epoch 214/500
90/90 [==============================] - 0s - loss: 319.8473     
Epoch 215/500
90/90 [==============================] - 0s - loss: 315.6953     
Epoch 216/500
90/90 [==============================] - 0s - loss: 311.6634     
Epoch 217/500
90/90 [==============================] -

90/90 [==============================] - 0s - loss: 86.4971     
Epoch 308/500
90/90 [==============================] - 0s - loss: 85.3629     
Epoch 309/500
90/90 [==============================] - 0s - loss: 83.9871     
Epoch 310/500
90/90 [==============================] - 0s - loss: 82.5688     
Epoch 311/500
90/90 [==============================] - 0s - loss: 81.5548     
Epoch 312/500
90/90 [==============================] - 0s - loss: 80.0724     
Epoch 313/500
90/90 [==============================] - 0s - loss: 78.8535     
Epoch 314/500
90/90 [==============================] - 0s - loss: 77.7450     
Epoch 315/500
90/90 [==============================] - 0s - loss: 76.4332     
Epoch 316/500
90/90 [==============================] - 0s - loss: 75.2576     
Epoch 317/500
90/90 [==============================] - 0s - loss: 74.1483     
Epoch 318/500
90/90 [==============================] - 0s - loss: 73.2550     
Epoch 319/500
90/90 [==============================] - 0s - loss: 

90/90 [==============================] - 0s - loss: 16.2264     
Epoch 411/500
90/90 [==============================] - 0s - loss: 15.9181     
Epoch 412/500
90/90 [==============================] - 0s - loss: 15.8171     
Epoch 413/500
90/90 [==============================] - 0s - loss: 15.5938     
Epoch 414/500
90/90 [==============================] - 0s - loss: 15.2729     
Epoch 415/500
90/90 [==============================] - 0s - loss: 15.1465     
Epoch 416/500
90/90 [==============================] - 0s - loss: 15.0433     
Epoch 417/500
90/90 [==============================] - 0s - loss: 14.6670     
Epoch 418/500
90/90 [==============================] - 0s - loss: 14.2541     
Epoch 419/500
90/90 [==============================] - 0s - loss: 14.4070     
Epoch 420/500
90/90 [==============================] - 0s - loss: 13.9839     
Epoch 421/500
90/90 [==============================] - 0s - loss: 13.5483     
Epoch 422/500
90/90 [==============================] - 0s - loss: 

In [26]:
test_loss = np.nan
if lstm_stateful:
    test_loss = model.evaluate(X_test, y_test, batch_size = batch_size)
else:
    test_loss = model.evaluate(X_test, y_test, batch_size = batch_size)
test_loss

 1/10 [==>...........................] - ETA: 0s

0.14562789704650642

In [27]:
if lstm_stateful:
    model.reset_states()
    pred_train = model.predict(X_train, batch_size = batch_size)
    model.reset_states()
    pred_test = model.predict(X_test, batch_size = batch_size)
else:
    pred_train = model.predict(X_train, batch_size = batch_size)
    pred_test = model.predict(X_test, batch_size = batch_size)

In [28]:
for i in X_test:
    if lstm_stateful:
        model.reset_states()
    #print(i)
    r = i.reshape(1, len(i), 1)
    #print(i.shape), print(r.shape)
    print(model.predict(r))

[[[ 26.04081154]
  [ 27.08862877]
  [ 28.02630615]
  [ 29.15218353]
  [ 30.31283951]]]
[[[ 27.14068222]
  [ 28.11089897]
  [ 29.06536865]
  [ 30.18508148]
  [ 31.34586906]]]
[[[ 28.25746918]
  [ 29.14868546]
  [ 30.1166153 ]
  [ 31.22247887]
  [ 32.37998962]]]
[[[ 29.3720665 ]
  [ 30.19871902]
  [ 31.17583466]
  [ 32.26210785]
  [ 33.41367722]]]
[[[ 30.4692421 ]
  [ 31.25772476]
  [ 32.2390976 ]
  [ 33.3019371 ]
  [ 34.44534302]]]
[[[ 31.53915024]
  [ 32.32231903]
  [ 33.30286789]
  [ 34.34003067]
  [ 35.47330475]]]
[[[ 32.57761383]
  [ 33.38915253]
  [ 34.36410522]
  [ 35.37451935]
  [ 36.49593353]]]
[[[ 33.58545685]
  [ 34.45514679]
  [ 35.42034912]
  [ 36.40371704]
  [ 37.51177979]]]
[[[ 34.56735992]
  [ 35.51779556]
  [ 36.46977997]
  [ 37.42622375]
  [ 38.51980209]]]
[[[ 35.53060532]
  [ 36.57525635]
  [ 37.51119232]
  [ 38.44110489]
  [ 39.51946259]]]


In [29]:
pred_test

array([[[ 26.04081154],
        [ 27.08862877],
        [ 28.02630615],
        [ 29.15218353],
        [ 30.31283951]],

       [[ 27.14068222],
        [ 28.11089897],
        [ 29.06536865],
        [ 30.18508148],
        [ 31.34586906]],

       [[ 28.25746918],
        [ 29.14868546],
        [ 30.1166153 ],
        [ 31.22247887],
        [ 32.37998962]],

       [[ 29.3720665 ],
        [ 30.19871902],
        [ 31.17583466],
        [ 32.26210785],
        [ 33.41367722]],

       [[ 30.4692421 ],
        [ 31.25772476],
        [ 32.2390976 ],
        [ 33.3019371 ],
        [ 34.44534302]],

       [[ 31.53915024],
        [ 32.32231903],
        [ 33.30286789],
        [ 34.34003067],
        [ 35.47330475]],

       [[ 32.57761383],
        [ 33.38915253],
        [ 34.36410522],
        [ 35.37451935],
        [ 36.49593353]],

       [[ 33.58545685],
        [ 34.45514679],
        [ 35.42034912],
        [ 36.40371704],
        [ 37.51177979]],

       [[ 34.56735992],


In [30]:
def calc_dependent_predictions(model, data, prediction_window):
    prediction_seqs = []
    for i in range(int(len(data)/prediction_window)):
        print('Calculating predictions starting from: {}'.format(i))
        curr_frame = data[i*prediction_window]
        predicted = []
        for j in range(prediction_window):
            #print('Calculating single prediction: {}'.format(j))
            #print(curr_frame)
            pred = model.predict(curr_frame[np.newaxis,:,:])[0,0]
            #pred = model.predict(curr_frame.reshape(1, len(curr_frame), 1)) # same
            #print(pred)
            predicted.append(pred)
            curr_frame = curr_frame[1:] 
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [31]:
prediction_window = 5 

prediction_seqs_train = calc_dependent_predictions(model, X_train, prediction_window)
prediction_seqs_test = calc_dependent_predictions(model, X_test, prediction_window)

Calculating predictions starting from: 0
Calculating predictions starting from: 1
Calculating predictions starting from: 2
Calculating predictions starting from: 3
Calculating predictions starting from: 4
Calculating predictions starting from: 5
Calculating predictions starting from: 6
Calculating predictions starting from: 7
Calculating predictions starting from: 8
Calculating predictions starting from: 9
Calculating predictions starting from: 10
Calculating predictions starting from: 11
Calculating predictions starting from: 12
Calculating predictions starting from: 13
Calculating predictions starting from: 14
Calculating predictions starting from: 15
Calculating predictions starting from: 16
Calculating predictions starting from: 17
Calculating predictions starting from: 0
Calculating predictions starting from: 1


In [32]:
prediction_seqs_train 

[[array([ 10.6394825], dtype=float32),
  array([ 10.20870495], dtype=float32),
  array([ 9.90982914], dtype=float32),
  array([ 9.79043388], dtype=float32),
  array([ 9.91140556], dtype=float32)],
 [array([ 10.3433857], dtype=float32),
  array([ 11.14436531], dtype=float32),
  array([ 12.31319618], dtype=float32),
  array([ 13.74533844], dtype=float32),
  array([ 15.25131226], dtype=float32)],
 [array([ 16.65025711], dtype=float32),
  array([ 17.85232544], dtype=float32),
  array([ 18.86245155], dtype=float32),
  array([ 19.73625946], dtype=float32),
  array([ 20.54011536], dtype=float32)],
 [array([ 21.33127594], dtype=float32),
  array([ 22.15166855], dtype=float32),
  array([ 23.02736092], dtype=float32),
  array([ 23.96984863], dtype=float32),
  array([ 24.97801399], dtype=float32)],
 [array([ 26.04081154], dtype=float32),
  array([ 27.14068222], dtype=float32),
  array([ 28.25746918], dtype=float32),
  array([ 29.3720665], dtype=float32),
  array([ 30.4692421], dtype=float32)],
 [

In [33]:
prediction_seqs_test

[[array([ 26.04081154], dtype=float32),
  array([ 27.14068222], dtype=float32),
  array([ 28.25746918], dtype=float32),
  array([ 29.3720665], dtype=float32),
  array([ 30.4692421], dtype=float32)],
 [array([ 31.53915024], dtype=float32),
  array([ 32.57761383], dtype=float32),
  array([ 33.58545685], dtype=float32),
  array([ 34.56735992], dtype=float32),
  array([ 35.53060532], dtype=float32)]]

In [34]:
y_train[:10]

array([[[  6.],
        [  7.],
        [  8.],
        [  9.],
        [ 10.]],

       [[  7.],
        [  8.],
        [  9.],
        [ 10.],
        [ 11.]],

       [[  8.],
        [  9.],
        [ 10.],
        [ 11.],
        [ 12.]],

       [[  9.],
        [ 10.],
        [ 11.],
        [ 12.],
        [ 13.]],

       [[ 10.],
        [ 11.],
        [ 12.],
        [ 13.],
        [ 14.]],

       [[ 11.],
        [ 12.],
        [ 13.],
        [ 14.],
        [ 15.]],

       [[ 12.],
        [ 13.],
        [ 14.],
        [ 15.],
        [ 16.]],

       [[ 13.],
        [ 14.],
        [ 15.],
        [ 16.],
        [ 17.]],

       [[ 14.],
        [ 15.],
        [ 16.],
        [ 17.],
        [ 18.]],

       [[ 15.],
        [ 16.],
        [ 17.],
        [ 18.],
        [ 19.]]])

In [35]:
pred_train[:10,0]

array([[ 10.6394825 ],
       [ 10.20870495],
       [  9.90982914],
       [  9.79043388],
       [  9.91140556],
       [ 10.3433857 ],
       [ 11.14436531],
       [ 12.31319618],
       [ 13.74533844],
       [ 15.25131226]], dtype=float32)

In [36]:
y_test[:10]

array([[[ 26.],
        [ 27.],
        [ 28.],
        [ 29.],
        [ 30.]],

       [[ 27.],
        [ 28.],
        [ 29.],
        [ 30.],
        [ 31.]],

       [[ 28.],
        [ 29.],
        [ 30.],
        [ 31.],
        [ 32.]],

       [[ 29.],
        [ 30.],
        [ 31.],
        [ 32.],
        [ 33.]],

       [[ 30.],
        [ 31.],
        [ 32.],
        [ 33.],
        [ 34.]],

       [[ 31.],
        [ 32.],
        [ 33.],
        [ 34.],
        [ 35.]],

       [[ 32.],
        [ 33.],
        [ 34.],
        [ 35.],
        [ 36.]],

       [[ 33.],
        [ 34.],
        [ 35.],
        [ 36.],
        [ 37.]],

       [[ 34.],
        [ 35.],
        [ 36.],
        [ 37.],
        [ 38.]],

       [[ 35.],
        [ 36.],
        [ 37.],
        [ 38.],
        [ 39.]]])

In [37]:
pred_test[:10,0]

array([[ 26.04081154],
       [ 27.14068222],
       [ 28.25746918],
       [ 29.3720665 ],
       [ 30.4692421 ],
       [ 31.53915024],
       [ 32.57761383],
       [ 33.58545685],
       [ 34.56735992],
       [ 35.53060532]], dtype=float32)

In [38]:
if scale:
    pred_train = scaler.inverse_transform(pred_train)
    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
    pred_test = scaler.inverse_transform(pred_test)
    y_test = scaler.inverse_transform(y_test.reshape(-1,1))


In [39]:
y_train[:10],pred_train[:10,0]

(array([[[  6.],
         [  7.],
         [  8.],
         [  9.],
         [ 10.]],
 
        [[  7.],
         [  8.],
         [  9.],
         [ 10.],
         [ 11.]],
 
        [[  8.],
         [  9.],
         [ 10.],
         [ 11.],
         [ 12.]],
 
        [[  9.],
         [ 10.],
         [ 11.],
         [ 12.],
         [ 13.]],
 
        [[ 10.],
         [ 11.],
         [ 12.],
         [ 13.],
         [ 14.]],
 
        [[ 11.],
         [ 12.],
         [ 13.],
         [ 14.],
         [ 15.]],
 
        [[ 12.],
         [ 13.],
         [ 14.],
         [ 15.],
         [ 16.]],
 
        [[ 13.],
         [ 14.],
         [ 15.],
         [ 16.],
         [ 17.]],
 
        [[ 14.],
         [ 15.],
         [ 16.],
         [ 17.],
         [ 18.]],
 
        [[ 15.],
         [ 16.],
         [ 17.],
         [ 18.],
         [ 19.]]]), array([[ 10.6394825 ],
        [ 10.20870495],
        [  9.90982914],
        [  9.79043388],
        [  9.91140556],
 

In [40]:
y_test[:],pred_test[:,0]

(array([[[ 26.],
         [ 27.],
         [ 28.],
         [ 29.],
         [ 30.]],
 
        [[ 27.],
         [ 28.],
         [ 29.],
         [ 30.],
         [ 31.]],
 
        [[ 28.],
         [ 29.],
         [ 30.],
         [ 31.],
         [ 32.]],
 
        [[ 29.],
         [ 30.],
         [ 31.],
         [ 32.],
         [ 33.]],
 
        [[ 30.],
         [ 31.],
         [ 32.],
         [ 33.],
         [ 34.]],
 
        [[ 31.],
         [ 32.],
         [ 33.],
         [ 34.],
         [ 35.]],
 
        [[ 32.],
         [ 33.],
         [ 34.],
         [ 35.],
         [ 36.]],
 
        [[ 33.],
         [ 34.],
         [ 35.],
         [ 36.],
         [ 37.]],
 
        [[ 34.],
         [ 35.],
         [ 36.],
         [ 37.],
         [ 38.]],
 
        [[ 35.],
         [ 36.],
         [ 37.],
         [ 38.],
         [ 39.]]]), array([[ 26.04081154],
        [ 27.14068222],
        [ 28.25746918],
        [ 29.3720665 ],
        [ 30.4692421 ],
 

In [41]:
# calculate root mean squared error
rsme_train = math.sqrt(mean_squared_error(y_train, pred_train[:,0]))
print('Train Score: %.2f RMSE' % (rsme_train))
rsme_test = math.sqrt(mean_squared_error(y_test, pred_test[:,0]))
print('Test Score: %.2f RMSE' % (rsme_test))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
print(len(ts_train), len(pred_train), len(y_train))
len(ts_test), len(pred_test), len(y_test) 

In [ ]:
# shift train predictions for plotting
pred_train_shifted = np.empty_like(ts_all)
print(pred_train_shifted.size)
pred_train_shifted[:, :] = np.nan
# train predictions start at position window_size + 1 (or window_size, if counting from 0)
pred_train_shifted[window_size : len(pred_train) + window_size, :] = pred_train
pred_train_shifted[:13]

In [ ]:
# shift test predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_test_shifted = np.empty_like(ts_all)
pred_test_shifted[:, :] = np.nan
pred_test_shifted[len(pred_train) + (window_size * 2) : len_overall + 1, :] = pred_test
pred_test_shifted[-13:]

In [ ]:
plt.plot(ts_all)
plt.plot(pred_train_shifted)
plt.plot(pred_test_shifted)
plt.savefig(testname + '_lstm_' + str(use_LSTM) + '_stateful_' + str(lstm_stateful) + '_window_' + str(window_size) +
            '_epochs_' + str(num_epochs) + '_2layers_' + str(lstm_stack_layers) + '_scale_' + str(scale) + '.png')
plt.show()

In [ ]:
plot_start = -30
plot_end = -1
plt.plot(ts_all[plot_start:plot_end])
plt.plot(pred_train_shifted[plot_start:plot_end])
plt.plot(pred_test_shifted[plot_start:plot_end])
plt.show()


In [ ]:
def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

In [ ]:
plot_results(pred_train, y_train)

In [ ]:
plot_results(pred_test, y_test)

In [ ]:
def plot_results_multiple(predicted_data, true_data, prediction_window):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_window)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()

In [ ]:
plot_results_multiple(prediction_seqs_train, y_test, prediction_window)

In [ ]:
plot_results_multiple(prediction_seqs_test, y_test, prediction_window)

In [ ]:
#seq2seq
# predict and append prediction to existing values
# TimeDistributedDense after return_sequences True

In [ ]:
# https://groups.google.com/forum/#!topic/keras-users/9GsDwkSdqBg